In [1]:
import sys
import os
import glob
import pathlib
import torch

import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from omegaconf import OmegaConf

from equivariant_pose_graph.dataset.point_cloud_data_module import MultiviewDataModule
from equivariant_pose_graph.utils.se3 import random_se3

def toDisplay(x, target_dim = 2):
    while(x.dim() > target_dim):
        x = x[0]
    return x.detach().cpu().numpy()


def plot_multi_np(plist):
    """
    Args: plist, list of numpy arrays of shape, (1,num_points,3)
    """
    colors = [
        '#1f77b4',  # muted blue
        '#ff7f0e',  # safety orange
        '#2ca02c',  # cooked asparagus green
        '#d62728',  # brick red
        '#9467bd',  # muted purple
        '#e377c2',  # raspberry yogurt pink
        '#8c564b',  # chestnut brown
        '#7f7f7f',  # middle gray
        '#bcbd22',  # curry yellow-green
        '#17becf'   # blue-teal
    ]
    skip = 1
    go_data = []
    for i in range(len(plist)):
        p_dp = toDisplay(torch.from_numpy(plist[i]))
        plot = go.Scatter3d(x=p_dp[::skip,0], y=p_dp[::skip,1], z=p_dp[::skip,2], 
                     mode='markers', marker=dict(size=2, color=colors[i],
                     symbol='circle'))
        go_data.append(plot)
 
    layout = go.Layout(
        scene=dict(
            aspectmode='data'
        )
    )

    fig = go.Figure(data=go_data, layout=layout)
    fig.show()
    return fig

cfg = OmegaConf.load("/home/odonca/workspace/rpad/equivariant_pose_graph/configs/data_augmentation/train_mug_residual_gradclip1e-3_axis_angle_uniform_z_2rackvariety.yaml")

dm = MultiviewDataModule(
    dataset_root=cfg.dataset_root,
    test_dataset_root=cfg.test_dataset_root,
    dataset_index=cfg.dataset_index,
    action_class=cfg.action_class,
    anchor_class=cfg.anchor_class,
    dataset_size=cfg.dataset_size,
    rotation_variance=np.pi/180 * cfg.rotation_variance,
    translation_variance=cfg.translation_variance,
    batch_size=cfg.batch_size,
    num_workers=cfg.num_workers,
    cloud_type=cfg.cloud_type,
    num_points=cfg.num_points,
    overfit=cfg.overfit,
    num_overfit_transforms=cfg.num_overfit_transforms,
    seed_overfit_transforms=cfg.seed_overfit_transforms,
    set_Y_transform_to_identity=cfg.set_Y_transform_to_identity,
    set_Y_transform_to_overfit=cfg.set_Y_transform_to_overfit,
    num_demo=cfg.num_demo,
    synthetic_occlusion=cfg.synthetic_occlusion,
    ball_radius=cfg.ball_radius,
    ball_occlusion=cfg.ball_occlusion,
    plane_occlusion=cfg.plane_occlusion,
    plane_standoff=cfg.plane_standoff,
    distractor_anchor_aug=cfg.distractor_anchor_aug,
    demo_mod_k_range=[cfg.demo_mod_k_range_min, cfg.demo_mod_k_range_max],
    demo_mod_rot_var=cfg.demo_mod_rot_var * np.pi/180,
    demo_mod_trans_var=cfg.demo_mod_trans_var,
    multimodal_transform_base=cfg.multimodal_transform_base,
    occlusion_class=cfg.occlusion_class,
    rot_sample_method=cfg.rot_sample_method,
    skip_failed_occlusion=cfg.skip_failed_occlusion,
    min_num_cameras=cfg.min_num_cameras,
    max_num_cameras=cfg.max_num_cameras,
)
dm.setup()

train_dataloader = dm.train_dataloader()
train_iter = iter(train_dataloader)

/home/odonca/workspace/rpad/.env_rpad_taxj/lib/python3.8/site-packages/torch_geometric/typing.py:47: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /lib/x86_64-linux-gnu/libm.so.6: version `GLIBC_2.29' not found (required by /home/odonca/workspace/rpad/.env_rpad_taxj/lib/python3.8/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/home/odonca/workspace/rpad/.env_rpad_taxj/lib/python3.8/site-packages/torch_geometric/typing.py:101: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /lib/x86_64-linux-gnu/libm.so.6: version `GLIBC_2.29' not found (required by /home/odonca/workspace/rpad/.env_rpad_taxj/lib/python3.8/site-packages/torch_sparse/_metis_cuda.so)
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


ConfigAttributeError: Missing key min_num_cameras
    full_key: min_num_cameras
    object_type=dict

In [ ]:
batch_0 = next(train_iter)

In [ ]:
from pytorch3d.transforms import matrix_to_euler_angles

rotation_matrices_axis_angle = []
rotation_matrices_axis_angle_uniform_z = []
N = 1000
for i in range(N):
    transform_axis_angle = random_se3(batch_0['points_anchor'].shape[0], rot_var=np.pi/180*180, trans_var=0, device=batch_0['points_anchor'].device, rot_sample_method='axis_angle')
    transform_axis_angle_uniform_z = random_se3(batch_0['points_anchor'].shape[0], rot_var=np.pi/180*180, trans_var=0, device=batch_0['points_anchor'].device, rot_sample_method='axis_angle_uniform_z')
    
    rotation_matrices_axis_angle.append(transform_axis_angle.get_matrix()[:, :3, :3])
    rotation_matrices_axis_angle_uniform_z.append(transform_axis_angle_uniform_z.get_matrix()[:, :3, :3])
    
rotation_matrices_axis_angle = torch.vstack([r for r in rotation_matrices_axis_angle])
rotation_matrices_axis_angle_uniform_z = torch.vstack([r for r in rotation_matrices_axis_angle_uniform_z])

euler_angles_axis_angle = matrix_to_euler_angles(rotation_matrices_axis_angle, convention='XYZ')
euler_angles_axis_angle_uniform_z = matrix_to_euler_angles(rotation_matrices_axis_angle_uniform_z, convention='XYZ')

euler_angles_axis_angle_np = euler_angles_axis_angle.numpy()
euler_angles_axis_angle_uniform_z_np = euler_angles_axis_angle_uniform_z.numpy()

# X-axis rotations
# plt.subplot(1, 3, 1)
# plt.hist(euler_angles_axis_angle_np[:, 0], bins=30)
# plt.title('Rotation about X-axis')

# # Y-axis rotations
# plt.subplot(1, 3, 2)
# plt.hist(euler_angles_axis_angle_np[:, 1], bins=30)
# plt.title('Rotation about Y-axis')

# # Z-axis rotations
# plt.subplot(1, 3, 3)
# plt.hist(euler_angles_axis_angle_np[:, 2], bins=30)
# plt.title('Rotation about Z-axis')

# plt.show()

fig, ax = plt.subplots(2, 3, figsize=(20, 8))
ax[0, 0].hist(euler_angles_axis_angle_np[:, 0], bins=30)
ax[0, 0].set_title('axis_angle: Rotation about X-axis')
ax[0, 1].hist(euler_angles_axis_angle_np[:, 1], bins=30)
ax[0, 1].set_title('axis_angle: Rotation about Y-axis')
ax[0, 2].hist(euler_angles_axis_angle_np[:, 2], bins=30)
ax[0, 2].set_title('axis_angle: Rotation about Z-axis')

ax[1, 0].hist(euler_angles_axis_angle_uniform_z_np[:, 0], bins=30)
ax[1, 0].set_title('axis_angle_uniform_z: Rotation about X-axis')
ax[1, 1].hist(euler_angles_axis_angle_uniform_z_np[:, 1], bins=30)
ax[1, 1].set_title('axis_angle_uniform_z: Rotation about Y-axis')
ax[1, 2].hist(euler_angles_axis_angle_uniform_z_np[:, 2], bins=30)
ax[1, 2].set_title('axis_angle_uniform_z: Rotation about Z-axis')

plt.show()
